# Notebook 2:  Clustering con GMM y Algoritmos Jerárquicos

En este notebook vamos a implementar el algoritmo de clustering de Mezcla de Normales en R. El objetivo es entender como funciona este algoritmo y  aprender a utilizarlo con un ejemplo simple. Se incluye además la estrategia para definir el número de grupos k. Al final se propone un ejercicio con nuevos datos, con el fin de aprender a utilizar el algoritmo. 

In [2]:
library(mclust)
library(cluster)
#ruta datos
#lee datos del archivo de texto como una matriz. La primera #columna se utiliza como vector de nombres de las filas
datos <-read.table("c:\\milk.dat",row.names=1,dec=".",header=TRUE)
summary(datos)

SyntaxError: keyword can't be an expression (<ipython-input-2-232abbe1504d>, line 5)

In [11]:
clus <-Mclust(datos,3)
summary(clus)
plot(clus)
clus




NameError: name 'clus' is not defined

In [2]:
cl <- clus$classification
centers <- t(clus$parameters$mean)
centers
cl


SyntaxError: invalid syntax (<ipython-input-2-933ec6add38a>, line 1)

**Ejercicio 1:**

a) Explique como funciona el método Mclust, describa los principales parámetros de entrada y salida.

b) Interprete y comente los resultados obtenidos. 

**Respuestas al ejercicio 1:**
    
a) El metodo mencionando se basa en Gaussian Mixtur e Models, los cuales son estimados por algoritmos EM. La funcionalidad 
es elegir automaticamente el mejor numero de componentes y parametros de covarianza, gracias a los criterios de informacion 
bayesianos.
- Como parametros de entrada, sera nuestros datos, y en este caso tambien asignando la cantidad de parametros(3), en lo cuales queremos clasificar nuestros dataset.
- Como parametros de salida tenemos el número de componentes los cuales modelan de manera óptima nuestros datos (esto si no lo hemos definido de manera manual) y la cantidad de elementos pertenecientes a dichos componentes, además de datos correspondientes a log-likelihood, n, df , BIC e ICL (explicados en la siguiente celda)


  - Call: La llamada coincidente (ejecutada).
  - Data: Matriz con datos de entrada.
  - ModelName: Un string que denota el modelo en el cual se produce el BIC óptimo.
  - n: Numero de observaciones en la data
  - d: dimension de la data
  - G: El numero optimo de 'Mixture components'
  - BIC: todos los valores BIC
  - loglik: 
  - df: numero de parametros estimados
  - bic: valores BIC del modelo seleccionado 
  - hypvol: parametros de hypervolume para el ruido de compoentes si es que requiere (sino, es null) 
  - parameters: proporción, media y varianza.
  - z: Una matriz la cual nos indica que para la entrada [i,k] la probabilidad de la observacion i en el data test pertenezca a  la clase k. 
  - classification: la clasificacion correspondiente a z.
  - uncertainty: la incertidumbre asociada con la clasificacion.

b) Podemos identificar que en este caso al haber seteado la cantidad de componentes en 3 al ejecutar el método obtendremos la mejor distribución posible de nuestros datos bajo estos parametros. Se puede apreciar en los gráficos los elementos pertenecientes a cada grupo, así mismo en la "Clsutering Table" la cantidad de elementos pertenecientes a cada componente.
    Luego con "centers" podemos apreciar la media de cada característica perteneciente a cada uno de los grupos. Tambien podemos apreciar en "cl" la clasificación en la cual se ha encasillado cada animal en el dataset .

In [ ]:
#Análisis de Componentes Principales para graficar los grupos obtenidos
#Calcula plano de componentes principales y grafica las observaciones con el número de su grupo
res <- princomp(datos, cor=TRUE)
res2 <- predict(res,centers)

biplot(res,xlabs=cl)
#grafica los centros de cada grupo en el plano de componentes principales utilizando colores distintos
points(res2[,1:2],col=1:9,pch=19)


**Criterio de Información Bayesiano**

Una forma de discriminar entre posibles agrupamientos, es calcular el Criterio de Información Bayesiano, cuya expresión genérica es:
$$BIC = log L(Datos\mid \hat{\theta}) - \frac{d}{2}log(N)$$
donde $d$ es el número de parámetros libres y $N$ el número de observaciones. A medida que aumenta el número de parámetros la verosimilitud aumenta, pero esto es moderado por el término en que depende de $d$.

Mclust calcula automática el BIC para distintas configuraciones de las matrices covarianza de cada Normal multivariada (que corresponden a distintos números de parámetros) así como distinto número de grupos. Por parsimonia, es recomendable escoger el primer número de grupos que representa un máximo local.

**Ejercicio 2:**

a) Utilice Mclust sin definir el número de grupos y considere el Criterio de información Bayesiano (BIC) para decidir el número de grupos óptimo. 

b) Discuta y compare  sus resultados con aquellos obtenidos con k-means

In [ ]:
clus2 <-Mclust(datos)
summary(clus2)
plot(clus2)

cl2 <- clus2$classification
centers2 <- t(clus2$parameters$mean)
centers2
cl2

En lo que sigue, utilizaremos el método hclust, que permite realizar análisis de agrupamiento jerárquico

In [ ]:
res3 <- hclust(dist(datos))
biplot(res, xlabs= cutree(res3,4))

**Ejercicio 3:** 

a) Explique como funciona el método hclust, describa parámetros de entrada y salida.

b) Interprete y comente los resultados obtenidos. 

c) Utilice otros parámetros en hclust y discuta como varían los resultados. 

**Ejercicio 4:** (3 puntos)

a) Considere los datos comunas.csv y escoja las comunas de una sola región para realizar sus análisis. Realice un análisis de clustering con K-means, GMM y Análisis Jerárquico para dichos datos, excluyendo el índice IDSE. Interprete sus resultados.

b) Estudie las medidas de calidad propuestas en Murphy, Capítulo 25: purity, rand index y mutual information (pdf en siveducMD). Calcule alguna de las medidas de calidad estudiadas, para cada uno de los análisis realizado en (a), considerando  una categorización (IDSE_bis) de los datos según el índice IDSE, de la manera siguiente:
1. IDSE_bis = 1 si 0 <= IDSE < 0.3
2. IDSE_bis = 2 si 0.3 <= IDSE < 0.4
3. IDSE_bis = 3 si 0.4 <= IDSE < 0.5
4. IDSE_bis = 4 si 0.5 <= IDSE < 0.6
5. IDSE_bis = 5 si 0.6 <= IDSE < 0.7
6. IDSE_bis = 6 si 0.7 <= IDSE < 0.8
7. IDSE_bis = 7 si 0.8 <= IDSE < 0.9
8. IDSE_bis = 8 si 0.9 <= IDSE < 1


Discuta sus resultados.